# 03 for comprehensions with either


Many of these examples where taken straight from the famous talk: "For: What is it good for?"<br>by Josh Suereth & Dick Wall

https://www.youtube.com/watch?v=WDaw2yXAa50&t=746s

Highly recommend to watch it!




*abstract sealed class Either[+A, +B]<br> extends Product<br>*<div><i>&nbsp;with scala.Serializable</i></div><div><br></div><div><b>Either represents a value of one of two possible types (a disjoint union). </b>An instance of Either is an instance of either scala.util.Left or scala.util.Right. <br></div><div><br></div><div>A common use of Either is as an alternative to scala.Option for dealing with possibly missing values. In this usage, scala.None is replaced with a scala.util.Left which can contain useful information. scala.util.Right takes the place of scala.Some. <br></div><div><br></div><div>Convention dictates that Left is used for failure and Right is used for success. For example, you could use Either[String, Int] to indicate whether a received input is a String or an Int.</div>

In [2]:
import scala.util.{Failure, Success, Try}


In [3]:
type ParseError = String
val e1: Either[ParseError, Int] = Right(1)
val e2: Either[ParseError, Int] = Right(5)

for {
  x <- e1.right
  y <- e2.right
} yield x * y


Right(5)

In [4]:
val e3: Either[ParseError, Int] = Left("Not a Number")
val e4: Either[ParseError, Int] = Right(5)

for {
  x <- e3.right
  y <- e4.right
} yield x * y

Left(Not a Number)

In [11]:
//Either is right biased since Scala 2.12, therefore we can omit the .right
for {
  x <- e3
  y <- e4
} yield x * y

Left(Not a Number)

In [6]:
//Why does this work?

for {
  x <- e1.right //flatMap
  y <- e2.right //map
} yield x * y

//Is the same as

e1.flatMap(
  x => e2.map(
    y => x * y)
)

Right(5)

In [7]:
e3.flatMap(
  x => e4.map(
    y => x * y)
)

Left(Not a Number)

*abstract sealed class Either[+A, +B]*

**<pre style="background-color:#ffffff;color:#000000;font-family:'JetBrains Mono';font-size:9.8pt;"><span style="color:#808080;font-style:italic;">/** Binds the given function across `Right</span><span style="background-color:#e2ffe2;">`</span><span style="color:#808080;font-style:italic;">.<br></span><span style="color:#808080;font-style:italic;"> *<br></span><span style="color:#808080;font-style:italic;"> *  </span><span style="font-weight:bold;">@param </span><span style="color:#808080;font-style:italic;">f The function to bind across `Right</span><span style="background-color:#e2ffe2;">`</span><span style="color:#808080;font-style:italic;">.<br></span><span style="color:#808080;font-style:italic;"> */<br></span><span style="color:#000080;font-weight:bold;">def </span>flatMap[<span style="color:#20999d;">A1 </span>&gt;: <span style="color:#20999d;">A</span>, <span style="color:#20999d;">B1</span>](f: <span style="color:#20999d;">B </span>=&gt; Either[<span style="color:#20999d;">A1</span>, <span style="color:#20999d;">B1</span>]): Either[<span style="color:#20999d;">A1</span>, <span style="color:#20999d;">B1</span>] = <span style="color:#000080;font-weight:bold;">this match </span>{<br>  <span style="color:#000080;font-weight:bold;">case </span>Right(b) =&gt; f(b)<br>  <span style="color:#000080;font-weight:bold;">case </span>_        =&gt; <span style="color:#000080;font-weight:bold;">this</span>.asInstanceOf[Either[<span style="color:#20999d;">A1</span>, <span style="color:#20999d;">B1</span>]]<br>}<br><br><span style="color:#808080;font-style:italic;">/** The given function is applied if this is a `Right</span><span style="background-color:#e2ffe2;">`</span><span style="color:#808080;font-style:italic;">.<br></span><span style="color:#808080;font-style:italic;"> *<br></span><span style="color:#808080;font-style:italic;"> *  {{{<br></span><span style="color:#808080;font-style:italic;"> *  Right(12).map(x =&gt; "flower") // Result: Right("flower")<br></span><span style="color:#808080;font-style:italic;"> *  Left(12).map(x =&gt; "flower")  // Result: Left(12)<br></span><span style="color:#808080;font-style:italic;"> *  }}}<br></span><span style="color:#808080;font-style:italic;"> */<br></span><span style="color:#000080;font-weight:bold;">def </span>map[<span style="color:#20999d;">B1</span>](f: <span style="color:#20999d;">B </span>=&gt; <span style="color:#20999d;">B1</span>): Either[<span style="color:#20999d;">A</span>, <span style="color:#20999d;">B1</span>] = <span style="color:#000080;font-weight:bold;">this match </span>{<br>  <span style="color:#000080;font-weight:bold;">case </span>Right(b) =&gt; Right(f(b))<br>  <span style="color:#000080;font-weight:bold;">case </span>_        =&gt; <span style="color:#000080;font-weight:bold;">this</span>.asInstanceOf[Either[<span style="color:#20999d;">A</span>, <span style="color:#20999d;">B1</span>]]<br>}</pre>

In [8]:
//Either is very useful, when reading/parsing data
type Name = String
type Age = Int

case class Person(name: Name, age: Int)

def parseName(s: String): Either[ParseError, Name] =
  if (s.isEmpty)
    Left(s"'$s' is not a valid Name")
  else
    Right(s)

def parseAge(s: String): Either[ParseError, Age] =
  Try(s.toInt) match {
    case Failure(_) => Left(s"'$s' is not a valid Age")
    case Success(age) => Right(age)
  }

def parsePerson(inputName: String, inputAge: String): Either[ParseError, Person] = {
  for {
    name <- parseName(inputName)
    age <- parseAge(inputAge)
  } yield Person(name, age)
}

In [10]:
parsePerson("Sally", "25")

Right(Person(Sally,25))

In [12]:
parsePerson("", "25")


Left('' is not a valid Name)

In [13]:
parsePerson("Sally", "twenty eight")


Left('twenty eight' is not a valid Age)